In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pickle

In [2]:
with open('zebal_version2.pickle', 'rb') as f:
    df = pickle.load(f)

In [3]:
df = df[df['board']!='가입 인사 (등업 필수)']
df = df.reset_index()

In [4]:
df.shape

(28569, 13)

In [5]:
df['L1'] = df['type'].apply(lambda x: x[0].upper())
df['L2'] = df['type'].apply(lambda x: x[1].upper())
df['L3'] = df['type'].apply(lambda x: x[2].upper())
df['L4'] = df['type'].apply(lambda x: x[3].upper())

In [6]:
with open('tfidf.pickle', 'rb') as f:
    tfidf = pickle.load(f)
with open('tfidf_ft_cbow_avg.pickle', 'rb') as f:
    tfidf_ft_cbow = pickle.load(f)
with open('tfidf_ft_sg_avg.pickle', 'rb') as f:
    tfidf_ft_sg = pickle.load(f)
with open('tfidf_w2v_cbow_avg.pickle', 'rb') as f:
    tfidf_w2v_cbow = pickle.load(f)
with open('tfidf_w2v_sg_avg.pickle', 'rb') as f:
    tfidf_w2v_sg = pickle.load(f)

In [7]:
with open('all_embbeding_df_v2.pickle', 'rb') as f:
    em = pickle.load(f)

In [8]:
def make_matrix(ndf):
    target=[]
    for i in ndf:
        target.append(i)
    return pd.DataFrame(target)

In [9]:
d2v_dvow = make_matrix(em['doc2vec_PV_DBOW'])
d2v_dm   = make_matrix(em['doc2vec_PV_DMO'])
w2v_cbow = make_matrix(em['w2v_cbow_avg'])
w2v_sg   = make_matrix(em['w2v_skipgram_avg'])
ft_cbow  = make_matrix(em['ft_cbow_avg'])
ft_sg    = make_matrix(em['ft_skip_gram_avg'])
bert     = make_matrix(em['bert'])

In [10]:
print(tfidf.shape)
print(tfidf_w2v_cbow.shape)
print(tfidf_w2v_sg.shape)
print(tfidf_ft_cbow.shape)
print(tfidf_ft_sg.shape)
print(d2v_dvow.shape)
print(d2v_dm.shape)
print(w2v_cbow.shape)
print(w2v_sg.shape)
print(ft_cbow.shape)
print(ft_sg.shape)
print(bert.shape)

(28569, 9798)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 768)


In [11]:
print(tfidf.isnull().sum().max())
print(tfidf_w2v_cbow.isnull().sum().max())
print(tfidf_w2v_sg.isnull().sum().max())
print(tfidf_ft_cbow.isnull().sum().max())
print(tfidf_ft_sg.isnull().sum().max())
print(d2v_dvow.isnull().sum().max())
print(d2v_dm.isnull().sum().max())
print(w2v_cbow.isnull().sum().max())
print(w2v_sg.isnull().sum().max())
print(ft_cbow.isnull().sum().max())
print(ft_sg.isnull().sum().max())
print(bert.isnull().sum().max())

0
21
21
8
21
0
0
8
8
0
0
0


In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

y_t = df['L1']
#X_t = tfidf
X_t = tfidf_w2v_cbow
# X_t = tfidf_w2v_sg
# X_t = tfidf_ft_cbow
# X_t = tfidf_ft_sg
# X_t = d2v_dvow
# X_t = d2v_dm
# X_t = w2v_cbow
# X_t = w2v_sg
# X_t = ft_cbow
# X_t = ft_sg
# X_t = bert

X_t.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

print("shape of Y :"+str(y_t.shape))
print("shape of X :"+str(X_t.shape))

shape of Y :(28569,)
shape of X :(28569, 300)


In [16]:
X_train,X_test,Y_train,Y_test = train_test_split(X_t, y_t, test_size=.20, random_state=42)
print("shape of X Train :"+str(X_train.shape))
print("shape of X Test :"+str(X_test.shape))
print("shape of Y Train :"+str(Y_train.shape))
print("shape of Y Test :"+str(Y_test.shape))

shape of X Train :(22855, 300)
shape of X Test :(5714, 300)
shape of Y Train :(22855,)
shape of Y Test :(5714,)


In [18]:
for this_C in [1,5,10,25,50,100]:
    clf3 = SVC(kernel='rbf',C=this_C).fit(X_train,Y_train)
    clf3train = clf3.score(X_train,Y_train)
    clf3test  = clf3.score(X_test,Y_test)
    print("SVM for Non Linear \n C:{} Training Score : {:2f} Test Score : {:2f}\n".format(this_C,clf3train,clf3test))

c:\users\jinny\miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM for Non Linear 
 C:1 Training Score : 0.670838 Test Score : 0.680434



c:\users\jinny\miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM for Non Linear 
 C:5 Training Score : 0.671100 Test Score : 0.680434



c:\users\jinny\miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


...

SVM for Non Linear 
 C:25 Training Score : 0.671538 Test Score : 0.680784



c:\users\jinny\miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM for Non Linear 
 C:50 Training Score : 0.672282 Test Score : 0.680609



c:\users\jinny\miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM for Non Linear 
 C:100 Training Score : 0.673244 Test Score : 0.680784

